# EUC-NMF (Euclid Non-negative Matrix Factorization)

## 因子分解

In [ ]:
%%shell
git clone https://github.com/tky823/audio_source_separation.git

In [ ]:
%cd "/content/audio_source_separation/egs/nmf-example/euc-nmf"

In [ ]:
import sys
sys.path.append("../../../src")

In [ ]:
import numpy as np
import scipy.signal as ss
import soundfile as sf
import IPython.display as ipd
import matplotlib.pyplot as plt

In [ ]:
from algorithm.nmf import EUCNMF

In [ ]:
plt.rcParams['figure.dpi'] = 200

In [ ]:
fft_size, hop_size = 1024, 256

In [ ]:
x, sr = sf.read("../../../dataset/sample-song/sample-1_electric-guitar_8000.wav")

### 音源信号

In [ ]:
ipd.Audio(x, rate=sr)

In [ ]:
_, _, spectrogram = ss.stft(x, nperseg=fft_size, noverlap=fft_size-hop_size)
X = np.abs(spectrogram)**2

In [ ]:
n_basis = 6

### EUC-NMFの実行

In [ ]:
np.random.seed(111)
nmf = EUCNMF(n_basis=n_basis)

In [ ]:
basis, activation = nmf(X, iteration=100)

In [ ]:
plt.figure()
plt.plot(nmf.loss, color='black')
plt.show()

### 再構成信号

In [ ]:
X[X < 1e-12] = 1e-12

In [ ]:
Z = basis @ activation

ratio = np.sqrt(Z / X)

estimated_spectrogram = ratio * spectrogram
_, estimated_signal = ss.istft(estimated_spectrogram, nperseg=fft_size, noverlap=fft_size-hop_size)
estimated_signal = estimated_signal / np.abs(estimated_signal).max()
ipd.Audio(estimated_signal, rate=sr)

### NMF後の各信号

In [ ]:
for idx in range(n_basis):
    Z = basis[:, idx: idx+1] * activation[idx: idx+1, :]

    ratio = np.sqrt(Z / X)

    estimated_spectrogram = ratio * spectrogram
    _, estimated_signal = ss.istft(estimated_spectrogram, nperseg=fft_size, noverlap=fft_size-hop_size)
    estimated_signal = estimated_signal / np.abs(estimated_signal).max()
    display(ipd.Audio(estimated_signal, rate=sr))

In [ ]:
for idx in range(n_basis):
    estimated_spectrogram = basis[:, idx: idx + 1] @ activation[idx: idx + 1, :]

    estimated_power = np.abs(estimated_spectrogram)**2
    estimated_power[estimated_power < 1e-12] = 1e-12
    log_spectrogram = 10 * np.log10(estimated_power)

    plt.figure()
    plt.pcolormesh(log_spectrogram, cmap='jet')
    plt.show()

## ドメインパラメータ
EUC-NMFのコスト関数は以下で定義される．
\begin{align}
    \mathcal{L}
    = \sum_{i,j}\left|d_{ij} - \left(\sum_{k}t_{ik}v_{kj}\right)^{\frac{2}{p}}\right|^{2},
\end{align}
ただし，$d_{ij}$ は目的信号のスペクトログラムであり, $p$はドメインパラメータである．
デフォルト値：　$p=2$．

In [ ]:
domain = 1

In [ ]:
np.random.seed(111)
nmf = EUCNMF(n_basis=n_basis, domain=domain)

In [ ]:
basis, activation = nmf(X, iteration=100)

In [ ]:
plt.figure()
plt.plot(nmf.loss, color='black')
plt.show()

In [ ]:
Z = (basis @ activation)**(2 / domain)

ratio = np.sqrt(Z / X)

estimated_spectrogram = ratio * spectrogram
_, estimated_signal = ss.istft(estimated_spectrogram, nperseg=fft_size, noverlap=fft_size-hop_size)
estimated_signal = estimated_signal / np.abs(estimated_signal).max()
ipd.Audio(estimated_signal, rate=sr)

In [ ]:
for idx in range(n_basis):
    Z = (basis[:, idx: idx+1] * activation[idx: idx+1, :])**(2 / domain)

    ratio = np.sqrt(Z / X)

    estimated_spectrogram = ratio * spectrogram
    _, estimated_signal = ss.istft(estimated_spectrogram, nperseg=fft_size, noverlap=fft_size-hop_size)
    estimated_signal = estimated_signal / np.abs(estimated_signal).max()
    display(ipd.Audio(estimated_signal, rate=sr))

In [ ]:
for idx in range(n_basis):
    estimated_spectrogram = (basis[:, idx: idx + 1] @ activation[idx: idx + 1, :])**(2 / domain)

    estimated_power = np.abs(estimated_spectrogram)**2
    estimated_power[estimated_power < 1e-12] = 1e-12
    log_spectrogram = 10 * np.log10(estimated_power)

    plt.figure()
    plt.pcolormesh(log_spectrogram, cmap='jet')
    plt.show()